In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
import numpy as np
import random
from tqdm import tqdm
import os

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


batch_size = 64
learning_rate = 0.03
num_epochs = 80

epsilon = 8 / 255
alpha = 2 / 255

beta = 1.0

# gradually increasing attack strength of PGD attacks
pgd_schedule = {
    1: 1,
    5: 2,
    10: 3,
    20: 5
}

# AWP parameters to ensure robustness
awp_gamma = 0.002
awp_eps   = 0.005
awp_start_epoch = 10

transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

Using device: cuda


In [ ]:
CLEAN_ROOT = '/kaggle/input/imagenet100/train.X1'
VAL_ROOT   = '/kaggle/input/imagenet100/val.X'

train_dataset = torchvision.datasets.ImageFolder(
    root=CLEAN_ROOT,
    transform=transform_train
)

class_mapping = train_dataset.class_to_idx

# custom dataset to ensure consistent class mapping
class AlignedImageFolder(torchvision.datasets.ImageFolder):
    def find_classes(self, directory):
        return list(class_mapping.keys()), class_mapping

test_dataset = AlignedImageFolder(
    root=VAL_ROOT,
    transform=transform_test
)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [ ]:
# loading folder of images and giving binary labels
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        self.register_buffer("mean", torch.tensor(mean).view(1,3,1,1))
        self.register_buffer("std", torch.tensor(std).view(1,3,1,1))
    def forward(self, x):
        return (x - self.mean) / self.std

base_model = resnet18(weights="IMAGENET1K_V1")
base_model.fc = nn.Linear(base_model.fc.in_features, 100)

model = nn.Sequential(
    Normalize([0.485, 0.456, 0.406],
              [0.229, 0.224, 0.225]),
    base_model
).to(device)


In [ ]:

optimizer = optim.SGD(
    model.parameters(),
    lr=learning_rate,
    momentum=0.9,
    weight_decay=5e-4
)

scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer,
    milestones=[20, 35],
    gamma=0.1
)

# MART loss function
def mart_loss(model, x, y, num_steps):
    x_adv = x.detach() + 0.001 * torch.randn_like(x)

    for _ in range(num_steps):
        x_adv.requires_grad_()
        logits_adv = model(x_adv)
        loss = F.cross_entropy(logits_adv, y)
        grad = torch.autograd.grad(loss, x_adv)[0]
        x_adv = x_adv + alpha * torch.sign(grad)
        x_adv = torch.min(torch.max(x_adv, x - epsilon), x + epsilon)
        x_adv = torch.clamp(x_adv, 0, 1).detach()

    logits_clean = model(x)
    logits_adv = model(x_adv)

    prob_clean = F.softmax(logits_clean, dim=1)
    true_prob = prob_clean.gather(1, y.unsqueeze(1)).squeeze()

    ce_adv = F.cross_entropy(logits_adv, y)

    kl = F.kl_div(
        F.log_softmax(logits_adv, dim=1),
        prob_clean,
        reduction="none"
    ).sum(dim=1)

    mart = torch.mean(kl * (1 - true_prob))
    return ce_adv + beta * mart

# AWP implementation
class AWP:
    def __init__(self, model, gamma, eps):
        self.model = model
        self.gamma = gamma
        self.eps = eps
        self.backup = {}

    @torch.no_grad()
    def perturb(self):
        for n, p in self.model.named_parameters():
            if p.requires_grad and p.grad is not None and "weight" in n:
                norm = torch.norm(p.grad)
                if norm > 0:
                    self.backup[n] = p.data.clone()
                    p.data.add_(self.gamma * p.grad / (norm + 1e-8))
                    p.data.clamp_(
                        self.backup[n] - self.eps,
                        self.backup[n] + self.eps
                    )

    @torch.no_grad()
    def restore(self):
        for n, p in self.model.named_parameters():
            if n in self.backup:
                p.data.copy_(self.backup[n])
        self.backup = {}

awp = AWP(model, awp_gamma, awp_eps)


In [6]:
def evaluate(model, loader, adversarial=False, steps=10):
    model.eval()
    correct, total = 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        if adversarial:
            x_adv = x.clone()
            for _ in range(steps):
                x_adv.requires_grad_()
                loss = F.cross_entropy(model(x_adv), y)
                grad = torch.autograd.grad(loss, x_adv)[0]
                x_adv = x_adv + alpha * torch.sign(grad)
                x_adv = torch.min(torch.max(x_adv, x - epsilon), x + epsilon)
                x_adv = torch.clamp(x_adv, 0, 1).detach()
            out = model(x_adv)
        else:
            out = model(x)

        pred = out.argmax(1)
        correct += pred.eq(y).sum().item()
        total += y.size(0)

    return 100 * correct / total


In [7]:
print("Starting MART training (corrected)...")

for epoch in range(1, num_epochs + 1):
    model.train()

    num_steps = max(v for k, v in pgd_schedule.items() if epoch >= k)
    total_loss = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")

    for x, y in loop:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        loss = mart_loss(model, x, y, num_steps)
        loss.backward()

        if epoch >= awp_start_epoch:
            awp.perturb()
            loss_awp = mart_loss(model, x, y, num_steps)
            loss_awp.backward()
            awp.restore()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        loop.set_postfix(loss=loss.item())

    scheduler.step()

    clean_acc = evaluate(model, test_loader, False)
    adv_acc = evaluate(model, test_loader, True, steps=num_steps)

    print(
        f"Epoch {epoch:03d} | "
        f"Loss {total_loss / len(train_loader.dataset):.4f} | "
        f"Clean {clean_acc:.2f}% | "
        f"Adv {adv_acc:.2f}%"
    )

Starting MART training (corrected)...


Epoch 1/80: 100%|██████████| 508/508 [02:33<00:00,  3.30it/s, loss=0.814]


Epoch 001 | Loss 1.6140 | Clean 79.84% | Adv 63.20%


Epoch 2/80: 100%|██████████| 508/508 [02:33<00:00,  3.31it/s, loss=0.864]


Epoch 002 | Loss 1.0641 | Clean 83.12% | Adv 68.40%


...

Epoch 032 | Loss 1.1887 | Clean 80.00% | Adv 49.76%


Epoch 33/80: 100%|██████████| 508/508 [10:48<00:00,  1.28s/it, loss=1.24] 


Epoch 033 | Loss 1.1792 | Clean 79.52% | Adv 50.64%


Epoch 34/80:   6%|▌         | 31/508 [00:40<10:19,  1.30s/it, loss=1.23] 


KeyboardInterrupt: 

In [8]:
torch.save({
    "epoch": epoch,
    "model_state": model.state_dict(),
    "optimizer_state": optimizer.state_dict(),
    "scheduler_state": scheduler.state_dict()
}, "mart_awp_resnet18_imagenet100.pth")

print("Training completed and model saved!")

Training completed and model saved!
